# Parse International Census Data from United States Census Bureau 

Dataset gateway is [here](https://www.census.gov/data-tools/demo/idb/informationGateway.php), and the full dataset can be downloaded [here](https://www2.census.gov/programs-surveys/international-programs/about/idb/idbzip.zip), see [here](https://www.census.gov/programs-surveys/international-programs/about/idb/faq.html#par_textimage_2).

See also Google's [COVID-19 public datasets](https://console.cloud.google.com/marketplace/details/bigquery-public-datasets/covid19-public-data-program?_ga=2.118817228.-458258001.1586199431&pli=1)

In [ ]:
import pandas as pd
from os.path import join

## Download census data

In [ ]:
%sx wget -O ./site-data/census-data.zip https://www2.census.gov/programs-surveys/international-programs/about/idb/idbzip.zip

In [ ]:
%sx unzip ./site-data/census-data.zip -d ./site-data/census-data

## Get population size of every country of the world

In [ ]:
censusPath = join('site-data', 'census-data', 'idbzip')

In [ ]:
countryCodes = pd.read_csv(join(censusPath, 'IDBextCTYS.txt'), sep='|', 
            names=['countryCode','countryName','landArea'])

In [ ]:
countryCodes.head()

In [ ]:
population = pd.read_csv(join(censusPath, 'IDBext001.txt'), sep='|', 
                         names=['countryCode', 'year', 'midYearPopulation'])

In [ ]:
currentYear = pd.Timestamp.today().year

In [ ]:
populationCurrent = population[population.year == currentYear]

In [ ]:
populationCurrent.head()

In [ ]:
populationWorld = pd.merge(countryCodes, populationCurrent, on = 'countryCode', how='left')
populationWorld = populationWorld[['countryCode', 'countryName', 'midYearPopulation']]

In [ ]:
assert populationWorld['midYearPopulation'].isnull().sum() == 0

In [ ]:
populationWorld.head()

In [ ]:
populationWorld.shape

In [ ]:
populationWorld.to_csv(join('site-data','census-data', 'populationCountries.csv'), index=False)

## Get age & sex distribution of every country of the world

In [ ]:
ageSex = pd.read_csv(join(censusPath, 'IDBext094.txt'), sep='|', 
           names=['countryCode', 'year', 'totalFlag', 'startAge', 'isOpen', 'endAge', 
                  'midYearPopulationAll', 'midYearPopulationMale', 'midYearPopulationFemale'])

ageSex = ageSex[(ageSex.year == currentYear) & (ageSex.totalFlag == 'A')]

In [ ]:
ageSexWorld = pd.merge(countryCodes, ageSex, on = 'countryCode', how='left')

In [ ]:
ageSexWorld = ageSexWorld[['countryCode', 'countryName', 'startAge', 'isOpen', 
                                 'endAge', 'midYearPopulationMale', 'midYearPopulationFemale']]

In [ ]:
assert ageSexWorld['midYearPopulationMale'].isnull().sum() == 0

In [ ]:
assert ageSexWorld['midYearPopulationFemale'].isnull().sum() == 0

In [ ]:
ageSexWorld.head()

In [ ]:
ageSexWorld.shape

In [ ]:
ageSexWorld.to_csv(join('site-data','census-data', 'ageSexCountries.csv'), index=False)

## Clean up

In [ ]:
from os import remove
from shutil import rmtree

In [ ]:
remove(join('./site-data', 'census-data.zip'))

In [ ]:
remove('../.wget-hsts')

In [ ]:
rmtree(join('./site-data', 'census-data', 'idbzip'))

In [ ]:
!pwd

In [ ]:
!ls site-data/cen